## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
1. Sunny, Sunny, Sunny: 0.4*(0.6)* 0.6*(0.6)* 0.6*(0.6) = 0.031104
2. Rainy, Sunny, Sunny: 0.6*(0.1)* 0.3*(0.6)* 0.6*(0.6) = 0.003888
3. Rainy, Rainy, Sunny: 0.6*(0.1)* 0.7*(0.1)* 0.6*(0.6) = 0.0015119
4. Rainy, Rainy, Rainy: 0.6*(0.1)* 0.7*(0.1)* 0.7*(0.1) = 0.000294
5. Rainy, Sunny, Rainy: 0.6*(0.1)* 0.6*(0.6)* 0.7*(0.1) = 0.0015119
6. Sunny, Rainy, Sunny: 0.4*(0.6)* 0.7*(0.1)* 0.6*(0.6) = 0.0060479
7. Sunny, Sunny, Rainy: 0.4*(0.6)* 0.6*(0.6)* 0.7*(0.1) = 0.0060479
8. Sunny, Rainy, Rainy: 0.4*(0.6)* 0.7*(0.1)* 0.7*(0.1) = 0.0060479


最大機率為: Sunny, Sunny, Sunny
```

### 根據上述條件, 寫出Viterbi應用程式

In [35]:
# 觀察狀態 observations = ('dry', 'damp', 'soggy')  在過程中”可視"的狀態  
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry

# 隱藏狀態 states = ('sunny', 'rainy') 系統的真實狀態(Sunny，Rainy)所有的詞都有可能是這兩狀態的其中一種
states = ('sunny', 'rainy')

# 初始的狀態機率 start_probability = {'sunny': 0.4, 'rainy': 0.6}   
start_probability = {'sunny': 0.4, 'rainy': 0.6}

#  狀態轉移矩陣 transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},       
#                           'rainy': {'sunny':0.3, 'rainy':0.7}}
# 隱藏狀態到另外一個隱藏狀態的機率，因為隱藏狀態為兩種，因此此狀態矩陣為 4x4 的二維矩陣
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
                          
# 狀態發射矩陣 emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
#                         'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
# 隱藏狀態觀察到某一個觀察狀態的機率，此矩陣表示在某狀態states下是某個特定詞的機率 P(Obersverd, Status) = P(Status) * P(Observed | Status)                         
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

### Viterbi 實際上是使用動態規劃求解隱馬可夫模型預測，即使用動態規劃求解最大機率路徑。
因此可以從 t=1的時刻，開始計算在時刻 t 狀態 i 的所有路徑的最大概率，直到到達終點 t=T 時狀態為 i 的所有路徑的最大機率。

In [45]:
def viterbi(obs, states, start_p, trans_p, emit_p):

    # Viterbi紀錄每個states最大機率路徑
    V = [{}]
    # 紀錄每個states的狀態路徑
    path = {}

    # Initialize base cases (t == 0)， t = 0, 狀態為某 state 的機率
    # 初始狀態，取得第一個觀察值dry
    # 第一個states的發生機率為初始機率 * 隱藏狀態的發射機率
    for state in states: # state = sunny or rainy 
        V[0][state] = start_p[state] * emit_p[state][obs[0]]  
        #  第一圈：[sunny:0.4]*[sunny情況下dry的機率為0.6] ＝ [{'sunny': 0.24}]
        #  第二圈：[rainy:0.6]*[rainy情況下dry的機率為0.1] ＝ [{'rainy': 0.06}]

        # print(V)   # [{'sunny': 0.24}]
        # print(V[0]) # {'sunny': 0.24}
        # print(V[0][state]) # 0.24

        path[state] = [state]  
        #  第一圈：{'sunny': ['sunny']}
        #  第二圈：{'sunny': ['sunny'], 'rainy': ['rainy']}

        print(f'若第一天天氣為：{state}, 機率為：{V[0][state]}, 路徑為：{path[state]}')  
    print('=======================================================================================')


    # Run Viterbi for t > 0 時狀態為某 state 的機率，開始計算剩下的兩次觀察值dry
    for t in range(1,len(obs)):
        print(f'next obs={t}' )
        V.append({})
        newpath={}
        
        for curr_state in states:
            prob_list = []
            for pre_state in states:
                # print(V[t - 1][pre_state]) # 前一個機率 -> 取得機率值0.24
                # print(trans_p[pre_state]) # {'sunny': 0.6, 'rainy': 0.4}
                # print(trans_p[pre_state][cur_state]) # 狀態轉移矩陣機率->取得機率值0.06
                # print(emit_p[cur_state]) #{'dry': 0.6, 'damp': 0.3, 'soggy': 0.1}
                # print(emit_p[cur_state][obs[t]]) # 發射矩陣機率->取得機率值0.6
                curr_state_prob = V[t-1][pre_state] * trans_p[pre_state][curr_state] * emit_p[curr_state][obs[t]]
                prob_list.append((pre_state,curr_state_prob))
            V[t][curr_state] = max(prob_list)[1]
            newpath[curr_state] = path[max(prob_list)[0]] + [curr_state]
            print(f'若今天天氣：{curr_state}，較有可能為：{newpath[curr_state]}，機率為：{V[t][curr_state]}')
        print('=======================================================================================')
           
        # Don't need to remember the old paths
        path = newpath
    
    # 取最後一個較大機率的 V值，即為最大機率路徑的機率
    (prob, state) = max([(V[len(obs) - 1][final_state], final_state) for final_state in states])
    return (prob, path[state])  

In [44]:
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

若第一天天氣為：sunny, 機率為：0.24, 路徑為：['sunny']
若第一天天氣為：rainy, 機率為：0.06, 路徑為：['rainy']
next obs=1
若今天天氣：sunny，較有可能為：['sunny', 'sunny']，機率為：0.08639999999999999
若今天天氣：rainy，較有可能為：['sunny', 'rainy']，機率為：0.009600000000000001
next obs=2
若今天天氣：sunny，較有可能為：['sunny', 'sunny', 'sunny']，機率為：0.031103999999999993
若今天天氣：rainy，較有可能為：['sunny', 'sunny', 'rainy']，機率為：0.0034560000000000003


In [38]:
result

(0.031103999999999993, ['sunny', 'sunny', 'sunny'])